In [ ]:
#source
#https://github.com/CivilNet/Gemfield/blob/master/src/python/pascal_voc_xml2json/pascal_voc_xml2json.py

In [1]:
import xml.etree.ElementTree as ET
import os
import json

In [2]:
coco = dict()
coco['images'] = []
coco['type'] = 'instances'
coco['annotations'] = []
coco['categories'] = []

category_set = dict()
image_set = set()

category_item_id = 0
image_id = 20180000000
annotation_id = 0

In [3]:
print(coco)

{'images': [], 'type': 'instances', 'annotations': [], 'categories': []}


In [9]:
os.listdir()

['.ipynb_checkpoints', 'xmltojson.ipynb']

In [4]:
def addCatItem(name):
    global category_item_id
    category_item = dict()
    category_item['supercategory'] = 'none'
    category_item_id += 1
    category_item['id'] = category_item_id
    category_item['name'] = name
    coco['categories'].append(category_item)
    category_set[name] = category_item_id
    return category_item_id

def addImgItem(file_name, size):
    global image_id
    if file_name is None:
        raise Exception('Could not find filename tag in xml file.')
    if size['width'] is None:
        raise Exception('Could not find width tag in xml file.')
    if size['height'] is None:
        raise Exception('Could not find height tag in xml file.')
    image_id += 1
    image_item = dict()
    image_item['id'] = image_id
    image_item['file_name'] = file_name
    image_item['width'] = size['width']
    image_item['height'] = size['height']
    coco['images'].append(image_item)
    image_set.add(file_name)
    return image_id

def addAnnoItem(object_name, image_id, category_id, bbox):
    global annotation_id
    annotation_item = dict()
    annotation_item['segmentation'] = []
    seg = []
    #bbox[] is x,y,w,h
    #left_top
    seg.append(bbox[0])
    seg.append(bbox[1])
    #left_bottom
    seg.append(bbox[0])
    seg.append(bbox[1] + bbox[3])
    #right_bottom
    seg.append(bbox[0] + bbox[2])
    seg.append(bbox[1] + bbox[3])
    #right_top
    seg.append(bbox[0] + bbox[2])
    seg.append(bbox[1])

    annotation_item['segmentation'].append(seg)

    annotation_item['area'] = bbox[2] * bbox[3]
    annotation_item['iscrowd'] = 0
    annotation_item['ignore'] = 0
    annotation_item['image_id'] = image_id
    annotation_item['bbox'] = bbox
    annotation_item['category_id'] = category_id
    annotation_id += 1
    annotation_item['id'] = annotation_id
    coco['annotations'].append(annotation_item)

def parseXmlFiles(xml_path): 
    for f in os.listdir(xml_path):
        if not f.endswith('.xml'):
            continue
        
        bndbox = dict()
        size = dict()
        current_image_id = None
        current_category_id = None
        file_name = None
        size['width'] = None
        size['height'] = None
        size['depth'] = None

        xml_file = os.path.join(xml_path, f)
        print(xml_file)

        tree = ET.parse(xml_file)
        root = tree.getroot()
        if root.tag != 'annotation':
            raise Exception('pascal voc xml root element should be annotation, rather than {}'.format(root.tag))

        #elem is <folder>, <filename>, <size>, <object>
        for elem in root:
            current_parent = elem.tag
            current_sub = None
            object_name = None
            
            if elem.tag == 'folder':
                continue
            
            if elem.tag == 'filename':
                file_name = elem.text
                if file_name in category_set:
                    raise Exception('file_name duplicated')
                
            #add img item only after parse <size> tag
            elif current_image_id is None and file_name is not None and size['width'] is not None:
                if file_name not in image_set:
                    current_image_id = addImgItem(file_name, size)
                    print('add image with {} and {}'.format(file_name, size))
                else:
                    raise Exception('duplicated image: {}'.format(file_name)) 
            #subelem is <width>, <height>, <depth>, <name>, <bndbox>
            for subelem in elem:
                bndbox ['xmin'] = None
                bndbox ['xmax'] = None
                bndbox ['ymin'] = None
                bndbox ['ymax'] = None
                
                current_sub = subelem.tag
                if current_parent == 'object' and subelem.tag == 'name':
                    object_name = subelem.text
                    if object_name not in category_set:
                        current_category_id = addCatItem(object_name)
                    else:
                        current_category_id = category_set[object_name]

                elif current_parent == 'size':
                    if size[subelem.tag] is not None:
                        raise Exception('xml structure broken at size tag.')
                    size[subelem.tag] = int(subelem.text)

                #option is <xmin>, <ymin>, <xmax>, <ymax>, when subelem is <bndbox>
                for option in subelem:
                    if current_sub == 'bndbox':
                        if bndbox[option.tag] is not None:
                            raise Exception('xml structure corrupted at bndbox tag.')
                        bndbox[option.tag] = int(option.text)

                #only after parse the <object> tag
                if bndbox['xmin'] is not None:
                    if object_name is None:
                        raise Exception('xml structure broken at bndbox tag')
                    if current_image_id is None:
                        raise Exception('xml structure broken at bndbox tag')
                    if current_category_id is None:
                        raise Exception('xml structure broken at bndbox tag')
                    bbox = []
                    #x
                    bbox.append(bndbox['xmin'])
                    #y
                    bbox.append(bndbox['ymin'])
                    #w
                    bbox.append(bndbox['xmax'] - bndbox['xmin'])
                    #h
                    bbox.append(bndbox['ymax'] - bndbox['ymin'])
                    print('add annotation with {},{},{},{}'.format(object_name, current_image_id, current_category_id, bbox))
                    addAnnoItem(object_name, current_image_id, current_category_id, bbox )

In [10]:
 xml_path = '..'
os.listdir(xml_path)

['Adachi',
 'Chiba',
 'DamageTypeDef.pdf',
 'Ichihara',
 'Json_convert',
 'Muroran',
 'Nagakute',
 'Numazu',
 'sample_submissions.csv',
 'Sumida']

In [8]:
if __name__ == '__main__':
    xml_path = '.'#'Annotations'
    json_file = 'instances.json'
    parseXmlFiles(xml_path)
    json.dump(coco, open(json_file, 'w'))

.\000005.xml
add image with 000005.jpg and {'width': 500, 'height': 375, 'depth': 3}
add annotation with chair,20180000001,1,[263, 211, 61, 128]
add annotation with chair,20180000001,1,[165, 264, 88, 108]
add annotation with chair,20180000001,1,[5, 244, 62, 130]
add annotation with chair,20180000001,1,[241, 194, 54, 105]
add annotation with chair,20180000001,1,[277, 186, 35, 34]
.\000007.xml
add image with 000007.jpg and {'width': 500, 'height': 333, 'depth': 3}
add annotation with car,20180000002,2,[141, 50, 359, 280]
.\000009.xml
add image with 000009.jpg and {'width': 500, 'height': 375, 'depth': 3}
add annotation with horse,20180000003,3,[69, 172, 201, 158]
add annotation with person,20180000003,4,[150, 141, 79, 143]
add annotation with person,20180000003,4,[285, 201, 42, 130]
add annotation with person,20180000003,4,[258, 198, 39, 131]
.\000012.xml
add image with 000012.jpg and {'width': 500, 'height': 333, 'depth': 3}
add annotation with car,20180000004,2,[156, 97, 195, 173]
.\00

In [14]:
areas = ['Adachi','Chiba']
print(areas[0])

for area in areas:
    xml_path = '../'+area+'/'+'Annotations'
    print(xml_path)
    json_file = 'instances.json'
    parseXmlFiles(xml_path)
    json.dump(coco, open(json_file, 'w'))


Adachi
../Adachi/Annotations
../Adachi/Annotations\train_Adachi_00001.xml
add image with train_Adachi_00001.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D44,20180000001,1,[424, 361, 166, 185]
add annotation with D10,20180000001,2,[114, 432, 373, 111]
../Adachi/Annotations\train_Adachi_00002.xml
add image with train_Adachi_00002.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D01,20180000002,3,[113, 353, 147, 239]
add annotation with D01,20180000002,3,[318, 300, 137, 282]
../Adachi/Annotations\train_Adachi_00003.xml
add image with train_Adachi_00003.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D01,20180000003,3,[474, 344, 124, 149]
../Adachi/Annotations\train_Adachi_00004.xml
add image with train_Adachi_00004.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D01,20180000004,3,[362, 228, 162, 350]
add annotation with D00,20180000004,4,[130, 310, 269, 279]
../Adachi/Annotations\train_Ad

add image with train_Adachi_00054.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D00,20180000054,4,[107, 312, 130, 206]
add annotation with D00,20180000054,4,[7, 304, 146, 88]
../Adachi/Annotations\train_Adachi_00055.xml
add image with train_Adachi_00055.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D00,20180000055,4,[141, 415, 63, 152]
../Adachi/Annotations\train_Adachi_00056.xml
add image with train_Adachi_00056.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D11,20180000056,8,[235, 209, 195, 20]
add annotation with D00,20180000056,4,[239, 303, 47, 84]
../Adachi/Annotations\train_Adachi_00057.xml
add image with train_Adachi_00057.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D10,20180000057,2,[236, 344, 188, 41]
add annotation with D10,20180000057,2,[168, 489, 399, 72]
../Adachi/Annotations\train_Adachi_00058.xml
add image with train_Adachi_00058.jpg and {'width': 600, 'height': 6

add image with train_Adachi_00090.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D01,20180000090,3,[339, 333, 261, 241]
../Adachi/Annotations\train_Adachi_00091.xml
add image with train_Adachi_00091.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D00,20180000091,4,[197, 365, 69, 230]
../Adachi/Annotations\train_Adachi_00092.xml
add image with train_Adachi_00092.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D01,20180000092,3,[327, 145, 93, 306]
../Adachi/Annotations\train_Adachi_00093.xml
add image with train_Adachi_00093.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D01,20180000093,3,[10, 293, 184, 196]
add annotation with D01,20180000093,3,[31, 246, 242, 307]
../Adachi/Annotations\train_Adachi_00094.xml
add image with train_Adachi_00094.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D44,20180000094,1,[358, 354, 239, 145]
add annotation with D00,20180000094

add image with train_Adachi_00132.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D01,20180000132,3,[319, 215, 82, 342]
../Adachi/Annotations\train_Adachi_00133.xml
add image with train_Adachi_00133.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D01,20180000133,3,[187, 337, 77, 252]
../Adachi/Annotations\train_Adachi_00134.xml
add image with train_Adachi_00134.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D44,20180000134,1,[5, 202, 214, 186]
add annotation with D01,20180000134,3,[326, 162, 103, 384]
../Adachi/Annotations\train_Adachi_00135.xml
add image with train_Adachi_00135.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D00,20180000135,4,[219, 432, 72, 168]
../Adachi/Annotations\train_Adachi_00136.xml
add image with train_Adachi_00136.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D01,20180000136,3,[530, 268, 70, 74]
add annotation with D40,20180000136,7,

add image with train_Adachi_00180.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D43,20180000180,6,[4, 359, 593, 148]
../Adachi/Annotations\train_Adachi_00181.xml
add image with train_Adachi_00181.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D01,20180000181,3,[452, 409, 136, 150]
add annotation with D43,20180000181,6,[260, 329, 296, 49]
../Adachi/Annotations\train_Adachi_00182.xml
add image with train_Adachi_00182.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D43,20180000182,6,[3, 330, 597, 269]
../Adachi/Annotations\train_Adachi_00183.xml
add image with train_Adachi_00183.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D01,20180000183,3,[380, 356, 205, 236]
add annotation with D44,20180000183,1,[380, 356, 204, 238]
add annotation with D00,20180000183,4,[76, 412, 96, 133]
../Adachi/Annotations\train_Adachi_00184.xml
add image with train_Adachi_00184.jpg and {'width': 600, 'height':

...

add image with train_Chiba_00187.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D44,20180001507,1,[407, 470, 190, 90]
add annotation with D44,20180001507,1,[218, 479, 90, 117]
../Chiba/Annotations\train_Chiba_00188.xml
add image with train_Chiba_00188.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D44,20180001508,1,[431, 473, 168, 100]
add annotation with D10,20180001508,2,[337, 466, 22, 110]
add annotation with D00,20180001508,4,[388, 467, 112, 112]
../Chiba/Annotations\train_Chiba_00189.xml
add image with train_Chiba_00189.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D44,20180001509,1,[29, 440, 207, 157]
add annotation with D00,20180001509,4,[113, 456, 298, 136]
../Chiba/Annotations\train_Chiba_00190.xml
add image with train_Chiba_00190.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D44,20180001510,1,[434, 474, 164, 97]
../Chiba/Annotations\train_Chiba_00191.xml
add image with tr

add image with train_Chiba_00235.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D44,20180001555,1,[416, 471, 28, 19]
add annotation with D44,20180001555,1,[518, 541, 66, 46]
add annotation with D01,20180001555,3,[395, 460, 193, 128]
add annotation with D00,20180001555,4,[340, 491, 38, 46]
../Chiba/Annotations\train_Chiba_00236.xml
add image with train_Chiba_00236.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D20,20180001556,5,[1, 483, 595, 113]
../Chiba/Annotations\train_Chiba_00237.xml
add image with train_Chiba_00237.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D01,20180001557,3,[316, 473, 82, 104]
../Chiba/Annotations\train_Chiba_00238.xml
add image with train_Chiba_00238.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D00,20180001558,4,[230, 460, 30, 106]
../Chiba/Annotations\train_Chiba_00239.xml
add image with train_Chiba_00239.jpg and {'width': 600, 'height': 600, 'depth': N

add image with train_Chiba_00275.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D43,20180001595,6,[76, 470, 524, 68]
../Chiba/Annotations\train_Chiba_00276.xml
add image with train_Chiba_00276.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D00,20180001596,4,[350, 515, 88, 67]
../Chiba/Annotations\train_Chiba_00277.xml
add image with train_Chiba_00277.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D01,20180001597,3,[381, 442, 123, 90]
add annotation with D00,20180001597,4,[317, 443, 31, 57]
../Chiba/Annotations\train_Chiba_00278.xml
add image with train_Chiba_00278.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D43,20180001598,6,[38, 504, 558, 96]
../Chiba/Annotations\train_Chiba_00279.xml
add image with train_Chiba_00279.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D44,20180001599,1,[244, 457, 93, 127]
add annotation with D20,20180001599,5,[486, 488, 106, 

add image with train_Chiba_00312.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D01,20180001632,3,[137, 484, 127, 104]
../Chiba/Annotations\train_Chiba_00313.xml
add image with train_Chiba_00313.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D01,20180001633,3,[1, 483, 51, 60]
add annotation with D01,20180001633,3,[147, 464, 181, 116]
../Chiba/Annotations\train_Chiba_00314.xml
add image with train_Chiba_00314.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D43,20180001634,6,[123, 508, 476, 89]
../Chiba/Annotations\train_Chiba_00315.xml
add image with train_Chiba_00315.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D00,20180001635,4,[104, 498, 100, 74]
../Chiba/Annotations\train_Chiba_00316.xml
add image with train_Chiba_00316.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D01,20180001636,3,[421, 446, 173, 127]
../Chiba/Annotations\train_Chiba_00317.xml
add ima

add image with train_Chiba_00365.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D44,20180001685,1,[265, 536, 90, 49]
add annotation with D01,20180001685,3,[197, 488, 55, 40]
add annotation with D00,20180001685,4,[74, 492, 59, 83]
../Chiba/Annotations\train_Chiba_00366.xml
add image with train_Chiba_00366.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D44,20180001686,1,[425, 447, 175, 121]
add annotation with D44,20180001686,1,[177, 449, 175, 136]
add annotation with D00,20180001686,4,[309, 459, 92, 123]
../Chiba/Annotations\train_Chiba_00367.xml
add image with train_Chiba_00367.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D43,20180001687,6,[7, 549, 590, 48]
../Chiba/Annotations\train_Chiba_00368.xml
add image with train_Chiba_00368.jpg and {'width': 600, 'height': 600, 'depth': None}
add annotation with D01,20180001688,3,[16, 472, 59, 124]
../Chiba/Annotations\train_Chiba_00369.xml
add image with train_Chib